In [ ]:
from debug_model_comparison import debug_model_comparison

In [ ]:
debug_models(
        model1_path=None,  # Replace with path to Script 1 checkpoint
        model2_path=None,  # Replace with path to Script 2 checkpoint
        device='cuda' if torch.cuda.is_available() else 'cpu'
    )

In [1]:
!pwd

/home/alok/Desktop/antispoof_offline


In [3]:
!python compare_models_from_scratch.py



TESTING MODEL: maxvit_tiny_tf_224
MODEL COMPARISON - CREATING FROM SCRATCH

Device: cpu
Model: maxvit_tiny_tf_224

CREATING SCRIPT 1 MODEL

[Script 1] Creating model: maxvit_tiny_tf_224
model.safetensors: 100%|█████████████████████| 124M/124M [00:09<00:00, 13.0MB/s]
[Script 1] Backbone created with num_classes=0
[Script 1] Backbone output dimension: 512
[Script 1] Model construction complete

CREATING SCRIPT 2 MODEL
[Script 2] Simulating ModelManager.load_or_download_model()

[Script 2] Creating model: maxvit_tiny_tf_224
[Script 2] Initializing with backbone
[Script 2] Used forward_features, output shape: torch.Size([2, 512, 7, 7])
[Script 2] Final backbone dimension: 512
[Script 2] Model initialization complete

DETAILED MODEL COMPARISON

1. PARAMETER COUNT
----------------------------------------
Script 1:
  Total: 30,667,210
  Trainable: 30,667,210
  Frozen: 0

Script 2:
  Total: 31,180,210
  Trainable: 31,180,210
  Frozen: 0

Difference:
  Total: 513,000 ✗
  Trainable: 513,000 ✗


In [19]:
!pwd


/home/alok/Desktop/antispoof_offline


In [1]:
from stage2_training_old import Stage2Trainer as trainer1
from stage2_training import Stage2Trainer as trainer2
import torch

In [2]:
from jupyter_netease_trainer import JupyterNeteaseTrainer
trainer = JupyterNeteaseTrainer('.', '.')
config1 = trainer.create_stage2_config('.')

🚀 NetEase Trainer initialized
📁 Data root: .
💾 Output dir: .
🗄️  Model cache: ./model
🔧 Device: cpu
⚠️  No standard splits found. Available: ['stage2_training_old.py', 'model_tests.ipynb', 'dataAnalyser.ipynb', 'jupyter_netease_trainer.py', 'model_utils.py', 'complete_file_structure.md', 'stage1_training_old.py', 'logs', 'jupyter_netease_trainer (copy).py', 'wfas_dataset_analyzer.py', 'DataSplit.ipynb', 'Untitled.ipynb', 'netease_training_notebook.py', '.git', 'debugModels.ipynb', 'dataset_analysis', 'dataset_merger.py', 'compare_models_from_scratch.py', 'debug_model_comparison.py', 'checkpoints', 'netease_experiment_1', 'anaconda_projects', 'run_netease_pipeline.py', '.ipynb_checkpoints', 'training.ipynb', 'stage1_training.py', 'model', 'dataset_splitter.py', '__pycache__', '__init__.py', 'stage2_training.py', 'model_manager.py', 'wfas_dataset.py']
📦 Cached models:
  stage2: maxvit_base_tf_224
  stage1: convnext_base, convnextv2_base


In [5]:
t1= trainer1(config1)
t2= trainer2(config1)

2025-06-27 18:23:29,337 - INFO - Loading pretrained weights from Hugging Face hub (timm/maxvit_base_tf_224.in1k)
2025-06-27 18:23:29,654 - INFO - [timm/maxvit_base_tf_224.in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
2025-06-27 18:23:31,024 - INFO - Total parameters: 119,093,462
2025-06-27 18:23:31,024 - INFO - Trainable parameters: 119,093,462
2025-06-27 18:23:31,025 - INFO - Loading model from local directory: ./model/stage2/maxvit_base_tf_224
2025-06-27 18:23:32,999 - INFO - Loaded model weights from: ./model/stage2/maxvit_base_tf_224/pytorch_model.bin
2025-06-27 18:23:33,015 - INFO - Successfully loaded model from local cache
2025-06-27 18:23:34,031 - INFO - Total parameters: 119,093,462
2025-06-27 18:23:34,032 - INFO - Trainable parameters: 119,093,462


Used backbone forward, output shape: torch.Size([2, 768])
Final backbone dimension: 768


In [7]:
model1 = t1.model

In [8]:
model2 = t2.model

In [11]:
device = 'cpu'
def count_parameters_detailed(model, name):
    results = {
        'total': 0,
        'trainable': 0,
        'frozen': 0,
        'by_component': {}
    }
    
    for param_name, param in model.named_parameters():
        component = param_name.split('.')[0]
        if component not in results['by_component']:
            results['by_component'][component] = {
                'total': 0,
                'trainable': 0,
                'frozen': 0
            }
        
        param_count = param.numel()
        results['total'] += param_count
        results['by_component'][component]['total'] += param_count
        
        if param.requires_grad:
            results['trainable'] += param_count
            results['by_component'][component]['trainable'] += param_count
        else:
            results['frozen'] += param_count
            results['by_component'][component]['frozen'] += param_count
    
    return results

stats1 = count_parameters_detailed(model1, "Script 1")
stats2 = count_parameters_detailed(model2, "Script 2")

print(f"Script 1:")
print(f"  Total: {stats1['total']:,}")
print(f"  Trainable: {stats1['trainable']:,}")
print(f"  Frozen: {stats1['frozen']:,}")

print(f"\nScript 2:")
print(f"  Total: {stats2['total']:,}")
print(f"  Trainable: {stats2['trainable']:,}")
print(f"  Frozen: {stats2['frozen']:,}")

print(f"\nDifference:")
print(f"  Total: {stats2['total'] - stats1['total']:,} {'✓' if stats2['total'] == stats1['total'] else '✗'}")
print(f"  Trainable: {stats2['trainable'] - stats1['trainable']:,} {'✓' if stats2['trainable'] == stats1['trainable'] else '✗'}")
print(f"  Frozen: {stats2['frozen'] - stats1['frozen']:,} {'✓' if stats2['frozen'] == stats1['frozen'] else '✗'}")

# Component breakdown
print("\nBy Component:")
all_components = set(stats1['by_component'].keys()) | set(stats2['by_component'].keys())
for component in sorted(all_components):
    s1 = stats1['by_component'].get(component, {'total': 0, 'trainable': 0})
    s2 = stats2['by_component'].get(component, {'total': 0, 'trainable': 0})
    
    print(f"\n  {component}:")
    print(f"    Script 1: {s1['total']:,} total, {s1['trainable']:,} trainable")
    print(f"    Script 2: {s2['total']:,} total, {s2['trainable']:,} trainable")
    if s1['total'] != s2['total'] or s1['trainable'] != s2['trainable']:
        print(f"    MISMATCH! ✗")

# 2. Layer name comparison
print("\n2. LAYER NAME COMPARISON")
print("-"*40)

params1_names = set(dict(model1.named_parameters()).keys())
params2_names = set(dict(model2.named_parameters()).keys())

only_in_1 = params1_names - params2_names
only_in_2 = params2_names - params1_names

if only_in_1:
    print(f"\nLayers only in Script 1 ({len(only_in_1)}):")
    for name in sorted(only_in_1)[:5]:  # Show first 5
        print(f"  - {name}")
    if len(only_in_1) > 5:
        print(f"  ... and {len(only_in_1) - 5} more")

if only_in_2:
    print(f"\nLayers only in Script 2 ({len(only_in_2)}):")
    for name in sorted(only_in_2)[:5]:  # Show first 5
        print(f"  - {name}")
    if len(only_in_2) > 5:
        print(f"  ... and {len(only_in_2) - 5} more")

if not only_in_1 and not only_in_2:
    print("✓ Both models have identical layer names")

# 3. Check requires_grad status
print("\n3. REQUIRES_GRAD STATUS")
print("-"*40)

common_params = params1_names & params2_names
grad_mismatches = []

params1_dict = dict(model1.named_parameters())
params2_dict = dict(model2.named_parameters())

for param_name in common_params:
    grad1 = params1_dict[param_name].requires_grad
    grad2 = params2_dict[param_name].requires_grad
    
    if grad1 != grad2:
        grad_mismatches.append((param_name, grad1, grad2))

if grad_mismatches:
    print(f"Found {len(grad_mismatches)} parameters with different requires_grad:")
    for name, grad1, grad2 in grad_mismatches[:5]:
        print(f"  {name}: Script1={grad1}, Script2={grad2}")
    if len(grad_mismatches) > 5:
        print(f"  ... and {len(grad_mismatches) - 5} more")
else:
    print("✓ All common parameters have same requires_grad status")

# 4. Test forward pass
print("\n4. FORWARD PASS TEST")
print("-"*40)

model1.eval()
model2.eval()

with torch.no_grad():
    test_input = torch.randn(2, 3, 224, 224).to(device)
    
    try:
        logits1, features1 = model1(test_input)
        print(f"Script 1: Success")
        print(f"  Logits shape: {logits1.shape}")
        print(f"  Features shape: {features1.shape}")
    except Exception as e:
        print(f"Script 1: FAILED - {e}")
    
    try:
        logits2, features2 = model2(test_input)
        print(f"Script 2: Success")
        print(f"  Logits shape: {logits2.shape}")
        print(f"  Features shape: {features2.shape}")
    except Exception as e:
        print(f"Script 2: FAILED - {e}")

# 5. Check backbone specifics
print("\n5. BACKBONE ANALYSIS")
print("-"*40)

if hasattr(model1, 'backbone') and hasattr(model2, 'backbone'):
    print(f"Script 1 backbone type: {type(model1.backbone).__name__}")
    print(f"Script 2 backbone type: {type(model2.backbone).__name__}")
    
    # Check if backbones have different number of parameters
    backbone1_params = sum(p.numel() for p in model1.backbone.parameters())
    backbone2_params = sum(p.numel() for p in model2.backbone.parameters())
    
    print(f"\nBackbone parameters:")
    print(f"  Script 1: {backbone1_params:,}")
    print(f"  Script 2: {backbone2_params:,}")
    print(f"  Difference: {backbone2_params - backbone1_params:,}")
    
    # Check if backbone has a head/classifier
    if hasattr(model1.backbone, 'head'):
        print(f"\nScript 1 backbone.head: {model1.backbone.head}")
    if hasattr(model2.backbone, 'head'):
        print(f"Script 2 backbone.head: {model2.backbone.head}")
        
    if hasattr(model1.backbone, 'classifier'):
        print(f"\nScript 1 backbone.classifier: {model1.backbone.classifier}")
    if hasattr(model2.backbone, 'classifier'):
        print(f"Script 2 backbone.classifier: {model2.backbone.classifier}")

# # 6. Training test
# print("\n6. GRADIENT FLOW TEST")
# print("-"*40)

# model1.train()
# model2.train()

# # Simple gradient test
# test_input = torch.randn(4, 3, 224, 224).to(device)
# test_labels = torch.randint(0, 2, (4,)).to(device)

# for model_name, model in [("Script 1", model1), ("Script 2", model2)]:
#     model.zero_grad()
    
#     logits, features = model(test_input)
#     loss = F.cross_entropy(logits, test_labels)
#     loss.backward()
    
#     # Check which components have gradients
#     components_with_grad = {}
#     for name, param in model.named_parameters():
#         component = name.split('.')[0]
#         if component not in components_with_grad:
#             components_with_grad[component] = {'total': 0, 'with_grad': 0}
        
#         components_with_grad[component]['total'] += 1
#         if param.grad is not None and param.grad.abs().sum() > 0:
#             components_with_grad[component]['with_grad'] += 1
    
#     print(f"\n{model_name} gradient flow:")
#     for component, stats in components_with_grad.items():
#         ratio = stats['with_grad'] / stats['total'] * 100
#         status = '✓' if ratio == 100 else '✗'
#         print(f"  {component}: {stats['with_grad']}/{stats['total']} ({ratio:.0f}%) {status}")

Script 1:
  Total: 119,093,462
  Trainable: 119,093,462
  Frozen: 0

Script 2:
  Total: 119,093,462
  Trainable: 119,093,462
  Frozen: 0

Difference:
  Total: 0 ✓
  Trainable: 0 ✓
  Frozen: 0 ✓

By Component:

  backbone:
    Script 1: 118,698,708 total, 118,698,708 trainable
    Script 2: 118,698,708 total, 118,698,708 trainable

  classifier:
    Script 1: 1,026 total, 1,026 trainable
    Script 2: 1,026 total, 1,026 trainable

  feature_projection:
    Script 1: 393,728 total, 393,728 trainable
    Script 2: 393,728 total, 393,728 trainable

2. LAYER NAME COMPARISON
----------------------------------------
✓ Both models have identical layer names

3. REQUIRES_GRAD STATUS
----------------------------------------
✓ All common parameters have same requires_grad status

4. FORWARD PASS TEST
----------------------------------------
Script 1: Success
  Logits shape: torch.Size([2, 2])
  Features shape: torch.Size([2, 512])
Script 2: Success
  Logits shape: torch.Size([2, 2])
  Features sh

In [15]:
import timm
model = timm.create_model(model_name='maxvit_base_tf_224', pretrained=True, num_classes=0)

2025-06-27 17:34:13,803 - INFO - Loading pretrained weights from Hugging Face hub (timm/maxvit_base_tf_224.in1k)
2025-06-27 17:34:14,161 - INFO - [timm/maxvit_base_tf_224.in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.


In [21]:
model.eval()

MaxxVit(
  (stem): Stem(
    (conv1): Conv2dSame(3, 64, kernel_size=(3, 3), stride=(2, 2))
    (norm1): BatchNormAct2d(
      64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): GELUTanh()
    )
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (stages): Sequential(
    (0): MaxxVitStage(
      (blocks): Sequential(
        (0): MaxxVitBlock(
          (conv): MbConvBlock(
            (shortcut): Downsample2d(
              (pool): AvgPool2dSame(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0))
              (expand): Conv2d(64, 96, kernel_size=(1, 1), stride=(1, 1))
            )
            (pre_norm): BatchNormAct2d(
              64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
              (drop): Identity()
              (act): Identity()
            )
            (down): Identity()
            (conv1_1x1): Conv2d(64, 384, kernel_size=(1, 1), stride=(1, 1), bias=False)
    

In [31]:
import torch
dummy_input = torch.randn(1, 3, 224, 224)
backbone_output = model(dummy_input)
print(backbone_output.shape)
backbone_dim = backbone_output.shape[1]

torch.Size([1, 768])


In [ ]:
model